In [4]:
"""
    Use NFGS to solve optimization problem
        In this part, you can see two def: bfgs_newton and dfp_newton.
    Before you run the code, you need to know which function and how
    many unknowns paras you need to solve, then prepare the initial 
    values(x)、Primitive Function(f) and Jocobian Matrix(jacobian).
    And you can see a example in the last section.
"""

import numpy as np
from sympy import symbols


def bfgs_newton(f, x, n, iters):
    """
    BFGS method
        :param f: the Primitive Function needed to solve
        :param x: the Initial values(n dims)
        :param n: the Number of unknowns
        :param iters: Maximum number of Iterations
        :return: x value
    """
    # 步长。设为1才能收敛，小于1不能收敛
    learning_rate = 1
    # 初始化B正定矩阵
    B = np.eye(n)
    x_len = x.shape[0]
    # 一阶导g的第二范式的最小值（阈值）
    epsilon = 1e-5
    for i in range(1, iters):
        g = jacobian(f, x)
        if np.linalg.norm(g) < epsilon:
            break
        p = np.linalg.solve(B, g)
        # 更新x值
        x_new = x - p*learning_rate
        print("第" + str(i) + "次迭代后的结果为:", x_new)
        g_new = jacobian(f, x_new)
        y = g_new - g
        k = x_new - x
        y_t = y.reshape([x_len, 1])
        Bk = np.dot(B, k)
        k_t_B = np.dot(k, B)
        kBk = np.dot(np.dot(k, B), k)
        # 更新B正定矩阵。完全按照公式来计算
        B = B + y_t*y/np.dot(y, k) - Bk.reshape([x_len, 1]) * k_t_B / kBk
        x = x_new
    return x


def dfp_newton(f, x, n, iters):
    """
    DFP method
        :param f: the Primitive Function needed to solve
        :param x: the Initial values(n dims)
        :param n: the Number of unknowns
        :param iters: Maximum number of Iterations
        :return: x value
    """
    # 步长
    learning_rate = 1
    # 初始化B正定矩阵
    G = np.eye(n)
    x_len = x.shape[0]
    # 一阶导g的第二范式的最小值（阈值）
    epsilon = 1e-5
    for i in range(1, iters):
        g = jacobian(f, x)
        if np.linalg.norm(g) < epsilon:
            break
        p = np.dot(G, g)
        # 更新x值
        x_new = x - p * learning_rate
        print("第" + str(i) + "次迭代后的结果为:", x_new)
        g_new = jacobian(f, x_new)
        y = g_new - g
        k = x_new - x
        Gy = np.dot(G, y)
        y_t_G = np.dot(y, G)
        yGy = np.dot(np.dot(y, G), y)
        # 更新G正定矩阵
        G = G + k.reshape([x_len, 1]) * k / np.dot(k, y) - Gy.reshape([x_len, 1]) * y_t_G / yGy
        x = x_new
    return x


if __name__ == "__main__":
    ## 
    def jacobian(f, x):
        """
        Define A jocobian matrix
            Calculate the First Order Derivative of the Primitive Function
        :param f: the Primitive Function needed to solve
        :param x: Input values when Iterating
        :return: the vector of gradient
        """
        dfdx1 = 2*(x[0]-10)
        dfdx2 = 4*(x[1]-12)
        dfdx3 = 8*(x[2]-8)
        gradient = np.array([dfdx1, dfdx2, dfdx3], dtype=float)
        return gradient

    # First, define the elements of a three-dimensional vector
    x1 = symbols("x1")
    x2 = symbols("x2")
    x3 = symbols("x3")
    x = np.array([1, 1, 1], dtype=float)
    f = (x1-10)**2+2*(x2-12)**2+4*(x3-8)**2
    print(bfgs_newton(f, x, 3, 2000))
    # print(dfp_newton(f, x, 3, 1000))

第1次迭代后的结果为: [19. 45. 57.]
第2次迭代后的结果为: [18.28652664 29.07291936  0.62690008]
第3次迭代后的结果为: [ -3.89371938 -54.6216661   17.45526368]
第4次迭代后的结果为: [ 6.1165179  12.53615448  8.0536277 ]
第5次迭代后的结果为: [14.26277202 11.67931793  7.98933007]
第6次迭代后的结果为: [ 9.99987357 11.99952717  7.99914594]
第7次迭代后的结果为: [10.00000116 12.00000658  8.00001866]
第8次迭代后的结果为: [10.         12.00000001  8.        ]
[10.         12.00000001  8.        ]
第1次迭代后的结果为: [19. 45. 57.]
第2次迭代后的结果为: [16.72766685 26.79929838  1.64537383]
第3次迭代后的结果为: [  4.47621635 -29.07866996  14.81005912]
第4次迭代后的结果为: [ 6.54870749 12.37894803  8.00126005]
第5次迭代后的结果为: [13.46688907 11.90282066  7.96796237]
第6次迭代后的结果为: [ 9.99996528 11.99974993  7.99998457]
第7次迭代后的结果为: [10.00000004 12.00000051  8.00000026]
[10.00000004 12.00000051  8.00000026]
